In [4]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [6]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [7]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [8]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [9]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#         print(type(labels2Idx))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [10]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
#     train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
#     dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)


In [11]:
file_train = '/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_train.csv'
file_dev = '/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Id = train_dev_sentences(filetrain=file_train,filedev=file_dev, check=2)

In [13]:
dev_labels[:10]

[1, 0, 0, 1, 0, 0, 0, 0, 0, 1]

In [105]:
dev_sentences[0]

'Dann diskutiere es doch mit dir selbst.'

In [14]:
n_labels =2

In [15]:
def compile_model(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [18]:
def test_model(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels_earlier:'number of original labels if loading a pretrained model',
               filename_to_save_weigths,
               batch_size, 
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               labels2Idx,
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None,
                ):
    
#     f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(labels_earlier)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
                                     save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                          labels2Idx= labels2Idx,tokenize= tokenize), 
                steps_per_epoch= steps_per_epoch, epochs=1,
                validation_data = generator(filename ='/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv', 
                                            batch_size = batch_size, check = check_for_generator, 
                                           labels2Idx = labels2Idx, tokenize = tokenize),
                validation_steps = math.ceil(len(dev_labels) / batch_size),
                callbacks = [checkpoint]
            )

#             testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
#             for i in range(len(dev_sentences)):
#                 testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
#             results = model.predict(testset_features)


# #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
#             predLabels = results.argmax(axis=-1)
#             devLabels = devLabels
#             f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
#             r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
#             a = accuracy_score(devLabels, predLabels)
#             if max_f1 < f1:
#                 print("model saved. F1 is %f" %(f1))
#                 model.save(filename_to_save_weigths)
#                 max_f1 = f1
#                 max_p = p
#                 max_r = r
#                 max_a = a
#             text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
#             print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
#         to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
#         print(to_write)
#         f.write(to_write)
#         total_f1+=max_f1
#         total_prec+=max_p
#         total_acc+=max_a
#         total_recall+=max_r    
#         print("*****************************************************************************")
#     final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
#     print(final_text)
#     f.write(final_text)
#     f.close()




In [19]:
print(dev_sentences[1])

Nationalistisches Geseiere.


In [111]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 1
number_of_epochs = 10
millionpostcorpus_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_millionpostcorpus.txt' 
millionpostcorpus_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_millionpostcorpus.h5'
batch_size=32
millionpostcorpus_train_file='/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_train.csv'
tokenize = True
labels2Idx = labels2Id
f1_measure='binary'
pos_label=1
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [112]:
test_model(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=millionpostcorpus_pretraining_log, 
           labels_earlier = len(labels2Idx),
           filename_to_save_weigths=millionpostcorpus_pretraining_save_weigths,
           batch_size=batch_size,
           train_file=millionpostcorpus_train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=load_model_weights,
           tokenize = tokenize,
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           check_for_generator= check_for_generator)

Test 1/3
Epoch: 1
Epoch 1/1
132/132 [==============================] - 34s 261ms/step - loss: 0.4134 - acc: 0.8653


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.3877 - acc: 0.8670
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.829
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 216ms/step - loss: 0.3771 - acc: 0.8646
model saved. F1 is 0.019802
Test-Data: Prec: 0.333, Rec: 0.010, F1: 0.020, Acc: 0.829
Epoch: 4
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.3607 - acc: 0.8662
Test-Data: Prec: 0.100, Rec: 0.010, F1: 0.019, Acc: 0.817
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.3435 - acc: 0.8679
model saved. F1 is 0.117647
Test-Data: Prec: 0.333, Rec: 0.071, F1: 0.118, Acc: 0.819
Epoch: 6
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.3186 - acc: 0.8757
Test-Data: Prec: 0.174, Rec: 0.082, F1: 0.111, Acc: 0.779
Epoch: 7
Epoch 1/1
132/132 [==============================] - 29s 

Epoch: 1
Epoch 1/1
132/132 [==============================] - 36s 272ms/step - loss: 0.4128 - acc: 0.8658
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
132/132 [==============================] - 29s 217ms/step - loss: 0.3894 - acc: 0.8667
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.827
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.3738 - acc: 0.8653
Test-Data: Prec: 0.182, Rec: 0.020, F1: 0.037, Acc: 0.819
Epoch: 4
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.3623 - acc: 0.8667
Test-Data: Prec: 0.143, Rec: 0.020, F1: 0.036, Acc: 0.813
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.3383 - acc: 0.8703
Test-Data: Prec: 0.261, Rec: 0.061, F1: 0.099, Acc: 0.812
Epoch: 6
Epoch 1/1
132/132 [==============================] - 29s 220ms/step - loss: 0.3151 - acc: 0.8759
Test-Data: Prec: 0.233, Rec: 0.071, F1: 0.109, Acc: 0.803
Epoch: 7
Epoch 1

Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 268ms/step - loss: 0.4089 - acc: 0.8629
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 2
Epoch 1/1
132/132 [==============================] - 29s 217ms/step - loss: 0.3866 - acc: 0.8665
Test-Data: Prec: 0.000, Rec: 0.000, F1: 0.000, Acc: 0.831
Epoch: 3
Epoch 1/1
132/132 [==============================] - 30s 230ms/step - loss: 0.3749 - acc: 0.8660
Test-Data: Prec: 0.500, Rec: 0.010, F1: 0.020, Acc: 0.831
Epoch: 4
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.3632 - acc: 0.8681
Test-Data: Prec: 0.333, Rec: 0.020, F1: 0.038, Acc: 0.827
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.3402 - acc: 0.8686
Test-Data: Prec: 0.156, Rec: 0.051, F1: 0.077, Acc: 0.793
Epoch: 6
Epoch 1/1
132/132 [==============================] - 29s 219ms/step - loss: 0.3183 - acc: 0.8783
Test-Data: Prec: 0.253, Rec: 0.224, F1: 0.238, Acc: 0.756
Epoch: 7
Epoch 1

TypeError: bad operand type for unary +: 'str'

# Transfer learning now

In [113]:
n_labels =2

In [114]:
train_sentences, train_labels, dev_sentences, dev_labels = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [115]:
print(len(train_sentences))
print(dev_labels[:20])

4200
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [116]:
dev_sentences[0]

'@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land'

In [117]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
millionpostcorpus_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_millionpostcorpus.txt' 
millionpostcorpus_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_millionpostcorpus.h5'
batch_size=32
offlang_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_millionpostcorpus.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [118]:
test_model(generator=generator, train_sentences=train_sentences, devLabels=devLabels, number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs, filename_to_log=millionpostcorpus_tl_log, filename_to_save_weigths=millionpostcorpus_tl_save_weigths,
          batch_size=batch_size,train_file=offlang_train_file, f1_measure=f1_measure, pos_label=pos_label, load_model_weights=load_model_weights,
          model_weights_file = model_weights_file, nb_sequence_length=nb_sequence_length, nb_embedding_dims=nb_embedding_dims, check_for_generator= check_for_generator)

Test 1/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 36s 270ms/step - loss: 0.5763 - acc: 0.7079
model saved. F1 is 0.539683
Test-Data: Prec: 0.684, Rec: 0.446, F1: 0.540, Acc: 0.749
Epoch: 2
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.4885 - acc: 0.7633
model saved. F1 is 0.640816
Test-Data: Prec: 0.704, Rec: 0.588, F1: 0.641, Acc: 0.782
Epoch: 3
Epoch 1/1
132/132 [==============================] - 30s 225ms/step - loss: 0.4393 - acc: 0.7924
Test-Data: Prec: 0.713, Rec: 0.539, F1: 0.614, Acc: 0.776
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.4181 - acc: 0.8040
model saved. F1 is 0.662698
Test-Data: Prec: 0.705, Rec: 0.625, F1: 0.663, Acc: 0.790
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 219ms/step - loss: 0.3820 - acc: 0.8277
model saved. F1 is 0.684825
Test-Data: Prec: 0.713, Rec: 0.659, F1: 0.685, Acc: 0.800
Epoch: 6
Epoch 1/1
132/132 [==============================]

132/132 [==============================] - 28s 215ms/step - loss: 0.0385 - acc: 0.9846
Test-Data: Prec: 0.681, Rec: 0.712, F1: 0.696, Acc: 0.795
Epoch: 49
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.0324 - acc: 0.9903
Test-Data: Prec: 0.711, Rec: 0.655, F1: 0.682, Acc: 0.798
Epoch: 50
Epoch 1/1
132/132 [==============================] - 28s 216ms/step - loss: 0.0377 - acc: 0.9870
Test-Data: Prec: 0.705, Rec: 0.697, F1: 0.701, Acc: 0.803
*****************************************************************************
Test 3/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 37s 277ms/step - loss: 0.5752 - acc: 0.7057
Test-Data: Prec: 0.728, Rec: 0.442, F1: 0.550, Acc: 0.761
Epoch: 2
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.4888 - acc: 0.7616
Test-Data: Prec: 0.729, Rec: 0.573, F1: 0.642, Acc: 0.788
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.4487 - acc: 0.7888
Test

132/132 [==============================] - 28s 213ms/step - loss: 0.0310 - acc: 0.9884
Test-Data: Prec: 0.721, Rec: 0.659, F1: 0.689, Acc: 0.803
Epoch: 47
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.0327 - acc: 0.9882
Test-Data: Prec: 0.688, Rec: 0.727, F1: 0.707, Acc: 0.801
Epoch: 48
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.0278 - acc: 0.9893
Test-Data: Prec: 0.745, Rec: 0.644, F1: 0.691, Acc: 0.809
Epoch: 49
Epoch 1/1
132/132 [==============================] - 28s 212ms/step - loss: 0.0346 - acc: 0.9877
Test-Data: Prec: 0.730, Rec: 0.689, F1: 0.709, Acc: 0.813
Epoch: 50
Epoch 1/1
132/132 [==============================] - 28s 213ms/step - loss: 0.0262 - acc: 0.9905
Test-Data: Prec: 0.700, Rec: 0.708, F1: 0.704, Acc: 0.803
*****************************************************************************
Test 5/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 265ms/step - loss: 0.5834 - acc: 0.7019
Te

TypeError: bad operand type for unary +: 'str'

# gradual_unfreezing

In [20]:
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [21]:
print(dev_sentences[0])
print(len(dev_sentences))
print(len(dev_labels))

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
808
808


In [35]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
    
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model(2)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(2, activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
#         checkpoint = ModelCheckpoint(filename_to_save_weigths, monitor='val_acc',save_best_only = True, 
#                                      save_weights_only = True)


        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx,tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)
#     f.close()

In [37]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (16,18), (3,18), (1,18)]

strings =['suf_bu','all_unfreeze','gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, all_unfreeze, gradual_unfreezing]

In [ ]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))


    generator = sequential_generator
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 5
    number_of_epochs = 50
    labels2Id = labels2Idx
    millionpostcorpus_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_tl_millionpostcorpus_' +string+'.txt' 
    print("log file: %s" %(millionpostcorpus_tl_log))

    millionpostcorpus_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_millionpostcorpus_'+string+'.h5'
    print("save weights file: %s" %(millionpostcorpus_tl_save_weigths))
    batch_size=32
    millionpostcorpus_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
    f1_measure='binary'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_millionpostcorpus.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=3

    test_model_tl_unfreezing(generator=generator, 
               train_sentences=train_sentences, 
               devLabels=devLabels, 
               number_of_tests= number_of_tests,
               number_of_epochs=number_of_epochs, 
               filename_to_log=millionpostcorpus_tl_log, 
               labels2Idx = labels2Id,
               filename_to_save_weigths=millionpostcorpus_tl_save_weigths,
               batch_size=batch_size,
               unfreezing_strategy = strategy,       
               train_file=millionpostcorpus_tl_train_file, 
               f1_measure=f1_measure, 
               pos_label=pos_label, 
               load_model_weights=load_model_weights,
               model_weights_file = model_weights_file, 
               nb_sequence_length=nb_sequence_length, 
               nb_embedding_dims=nb_embedding_dims, 
               check_for_generator= check_for_generator)

approach: suf_bu
log file: /home/jindal/notebooks/jindal/NER/language_model/results_tl_millionpostcorpus_suf_bu.txt
save weights file: /home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_millionpostcorpus_suf_bu.h5
[(18, 18), (17, 16), (15, 3), (2, 1), (18, 1)]
Test 1/5
removing top layer
unfreezing dense_39
---------------------------------------
0 input_13 False
1 bidirectional_13 False
2 leaky_re_lu_61 False
3 conv1d_37 False
4 conv1d_38 False
5 conv1d_39 False
6 leaky_re_lu_62 False
7 leaky_re_lu_63 False
8 leaky_re_lu_64 False
9 global_max_pooling1d_37 False
10 global_max_pooling1d_38 False
11 global_max_pooling1d_39 False
12 dropout_37 False
13 dropout_38 False
14 dropout_39 False
15 concatenate_13 False
16 dense_37 False
17 leaky_re_lu_65 False
18 dense_39 True
Epoch: 1/50
Epoch 1/1
132/132 [==============================] - 14s 104ms/step - loss: 0.6403 - acc: 0.6368
model saved. F1 is 0.230088
Test-Data: Prec: 0.542, Rec: 0.146, F1: 0.230, Acc: 0.677
Epoch

132/132 [==============================] - 10s 73ms/step - loss: 0.6044 - acc: 0.6809
Test-Data: Prec: 0.557, Rec: 0.165, F1: 0.254, Acc: 0.681
Epoch: 46/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.6036 - acc: 0.6738
Test-Data: Prec: 0.549, Rec: 0.187, F1: 0.279, Acc: 0.681
Epoch: 47/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.6074 - acc: 0.6747
Test-Data: Prec: 0.569, Rec: 0.124, F1: 0.203, Acc: 0.679
Epoch: 48/50
Epoch 1/1
132/132 [==============================] - 10s 72ms/step - loss: 0.6087 - acc: 0.6679
Test-Data: Prec: 0.549, Rec: 0.146, F1: 0.231, Acc: 0.678
Epoch: 49/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.6040 - acc: 0.6761
Test-Data: Prec: 0.538, Rec: 0.157, F1: 0.243, Acc: 0.677
Epoch: 50/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.6053 - acc: 0.6745
Test-Data: Prec: 0.507, Rec: 0.127, F1: 0.204, Acc: 0.671
unfreezing leaky_re_l

132/132 [==============================] - 10s 74ms/step - loss: 0.6026 - acc: 0.6870
Test-Data: Prec: 0.667, Rec: 0.270, F1: 0.384, Acc: 0.714
Epoch: 41/50
Epoch 1/1
132/132 [==============================] - 10s 75ms/step - loss: 0.6040 - acc: 0.6799
Test-Data: Prec: 0.681, Rec: 0.240, F1: 0.355, Acc: 0.712
Epoch: 42/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.5934 - acc: 0.7003
Test-Data: Prec: 0.664, Rec: 0.273, F1: 0.387, Acc: 0.714
Epoch: 43/50
Epoch 1/1
132/132 [==============================] - 10s 73ms/step - loss: 0.5915 - acc: 0.6896
Test-Data: Prec: 0.654, Rec: 0.318, F1: 0.428, Acc: 0.719
Epoch: 44/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.6000 - acc: 0.6858
Test-Data: Prec: 0.670, Rec: 0.258, F1: 0.373, Acc: 0.713
Epoch: 45/50
Epoch 1/1
132/132 [==============================] - 10s 74ms/step - loss: 0.6039 - acc: 0.6731
Test-Data: Prec: 0.687, Rec: 0.213, F1: 0.326, Acc: 0.708
Epoch: 46/50
Epoch 1/

132/132 [==============================] - 30s 227ms/step - loss: 0.0518 - acc: 0.9804
Test-Data: Prec: 0.689, Rec: 0.704, F1: 0.696, Acc: 0.797
Epoch: 31/50
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.0570 - acc: 0.9785
Test-Data: Prec: 0.736, Rec: 0.625, F1: 0.676, Acc: 0.802
Epoch: 32/50
Epoch 1/1
132/132 [==============================] - 30s 231ms/step - loss: 0.0553 - acc: 0.9792
Test-Data: Prec: 0.723, Rec: 0.558, F1: 0.630, Acc: 0.783
Epoch: 33/50
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0532 - acc: 0.9813
Test-Data: Prec: 0.718, Rec: 0.581, F1: 0.642, Acc: 0.786
Epoch: 34/50
Epoch 1/1
132/132 [==============================] - 30s 228ms/step - loss: 0.0457 - acc: 0.9839
Test-Data: Prec: 0.744, Rec: 0.577, F1: 0.650, Acc: 0.795
Epoch: 35/50
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.0582 - acc: 0.9777
Test-Data: Prec: 0.706, Rec: 0.622, F1: 0.661, Acc: 0.790
Epoch: 36/50
Ep